In [1]:
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array
import tkinter as tk
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display


In [2]:
DRAWING_AREA = (280, 280)

def simple_cnn_preprocessor(data):
    return np.expand_dims(data, axis=0)

predictions_digit = [0 for x in range(10)]
predictions_alpha = [0 for x in range(26)]

ax = None

In [3]:
print(len(predictions_digit))
print(len(predictions_alpha))

10
26


In [4]:
# load the model
with open('./model/model_alpha_small.json', 'r') as json_file:
    model_alpha_json = json_file.read()
model_alpha = model_from_json(model_alpha_json)
model_alpha.load_weights('./model/model_alpha_small.h5')

In [5]:
# load the model
with open('./model/model_digit.json', 'r') as json_file:
    model_json = json_file.read()
model_digit= model_from_json(model_json)
model_digit.load_weights('./model/model_digit.h5')

In [6]:
def new_image():
    return Image.new("L", DRAWING_AREA)

In [8]:

image = new_image()
drawer = ImageDraw.Draw(image)

def process_image():
    # Load the image
    temp_image = image.copy()  

    # Convert the image to grayscale
    gray_image = temp_image.convert('L')
    gray_image = gray_image.resize((28, 28))
    display(gray_image)
    temp_image = img_to_array(gray_image)
    temp_image = simple_cnn_preprocessor(temp_image)
    temp_image = temp_image / 255
    return temp_image

def digit_predict():
    temp_image=process_image()
    predictions_digit = model_digit.predict(temp_image)[0]
    max_index, max_element = max(enumerate(predictions_digit), key=lambda x: x[1])
    return max_index

def alphabet_predict():
    temp_image=process_image()
    predictions_alpha = model_alpha.predict(temp_image)[0].tolist()
    max_index, max_element = max(enumerate(predictions_alpha), key=lambda x: x[1])
    return max_index

def predict_and_display():
    selected_option = var.get()  # Get the selected option (Digit or Alphabet)                    
    result = predict_img(selected_option)

    # Show the result in the text field
    text_field.config(state=tk.NORMAL)
    text_field.delete(1.0, tk.END)
    text_field.insert(tk.END, result)
    text_field.config(state=tk.DISABLED)


def predict_img(selected_option):

    if selected_option == "Digit":
        result="The digit is: "
        pre=digit_predict()

        pre=str(pre)
        result+=pre  
        print(result)       
        return result
    
    elif selected_option == "Alphabet":
        result="The Alphabet is: "
        pre=alphabet_predict()
        new=chr(ord('A') + pre )
        result+=new        
        print(result)
        return result
    else:
        return "Select Digit or Alphabet"

# helper functions
def _paint_(event):
    global predictions, ax
    x1, y1 = (event.x - 5), (event.y - 5)
    x2, y2 = (event.x + 5), (event.y + 5)
    canvas.create_oval(x1, y1, x2, y2, fill="#FFFFFF", outline="")
    drawer.ellipse([x1, y1, x2, y2], fill=255, outline=255)

def _clear_():
    global image, drawer, predictions
    canvas.delete("all")
    image = new_image()
    drawer = ImageDraw.Draw(image)
    text_field.config(state=tk.NORMAL)
    text_field.delete(1.0, tk.END)

# create the canvas
master = tk.Tk()
master.title("Digit Alphabet Recognition")
canvas = tk.Canvas(master,
                   width=DRAWING_AREA[0],
                   height=DRAWING_AREA[1])
canvas.configure(background="black")
canvas.pack()
canvas.bind("<B1-Motion>", _paint_)

# Create a variable to store the selected option (Digit or Alphabet)
var = tk.StringVar()

# Checkboxes for selecting "Digit" and "Alphabet"
digit_checkbox = tk.Checkbutton(master, text="Digit", variable=var, onvalue="Digit", offvalue="")
alphabet_checkbox = tk.Checkbutton(master, text="Alphabet", variable=var, onvalue="Alphabet", offvalue="")
digit_checkbox.pack()
alphabet_checkbox.pack()

predict=tk.Button(master, text="Predict", command=predict_and_display)
predict.pack()

clear = tk.Button(master, text="CLEAR", command=_clear_)
clear.pack(side=tk.BOTTOM)

# Text field for displaying output
text_field = tk.Text(master, height=6, width=40, state=tk.DISABLED)
text_field.pack()

tk.mainloop()

1/1 [==============================] - 0s 21ms/step
The digit is: 3


1/1 [==============================] - 0s 20ms/step
The digit is: 8


1/1 [==============================] - 0s 21ms/step
The digit is: 3
